In [1]:
from pyspark.sql import SparkSession
import pyspark.sql as sparksql
spark = SparkSession.builder.appName('stroke').getOrCreate()
df = spark.read.csv('/FileStore/tables/train.csv', inferSchema=True,header=True)
df.printSchema()

root
-- id: integer (nullable = true)
-- gender: string (nullable = true)
-- age: double (nullable = true)
-- hypertension: integer (nullable = true)
-- heart_disease: integer (nullable = true)
-- ever_married: string (nullable = true)
-- work_type: string (nullable = true)
-- Residence_type: string (nullable = true)
-- avg_glucose_level: double (nullable = true)
-- bmi: double (nullable = true)
-- smoking_status: string (nullable = true)
-- stroke: integer (nullable = true)

In [2]:
df = df.na.fill('Unknown', subset=['smoking_status'])

In [3]:
import pandas
data = df.toPandas()
print(data.describe())
print(data.dtypes)
print('\nCount missing value of each column:')
print(data.isnull().sum())

id age hypertension heart_disease \
count 43400.000000 43400.000000 43400.000000 43400.000000 
mean 36326.142350 42.217894 0.093571 0.047512 
std 21072.134879 22.519649 0.291235 0.212733 
min 1.000000 0.080000 0.000000 0.000000 
25% 18038.500000 24.000000 0.000000 0.000000 
50% 36351.500000 44.000000 0.000000 0.000000 
75% 54514.250000 60.000000 0.000000 0.000000 
max 72943.000000 82.000000 1.000000 1.000000 

 avg_glucose_level bmi stroke 
count 43400.000000 43400.000000 43400.000000 
mean 104.482750 28.605038 0.018041 
std 43.111751 7.638023 0.133103 
min 55.000000 10.100000 0.000000 
25% 77.540000 23.400000 0.000000 
50% 91.580000 28.100000 0.000000 
75% 112.070000 32.600000 0.000000 
max 291.050000 97.600000 1.000000 
id int32
gender object
age float64
hypertension int32
heart_disease int32
ever_married object
work_type object
Residence_type object
avg_glucose_level float64
bmi float64
smoking_status object
stroke int32
dtype: object

Count missing value of each column:
id 0
gender 0
age 0
hypertension 0
heart_disease 0
ever_married 0
work_type 0
Residence_type 0
avg_glucose_level 0
bmi 0
smoking_status 0
stroke 0
dtype: int64

In [4]:
data.groupby('stroke').size()   #check whether is an imbalanced dataset

Out[ 4 ]: 
stroke
0 42617
1 783
dtype: int64

In [5]:
import sklearn
print(sklearn.__version__)

0.18.1

In [6]:
%sh
#need to run ***ONCE*** to install SMOTE package
/home/ubuntu/databricks/python/bin/pip install 'imbalanced-learn<0.2.1'
pip freeze | grep imbalanced-learn

Collecting imbalanced-learn<0.2.1
 Downloading https://files.pythonhosted.org/packages/5a/e0/e4d476839c4fd30325eecb62f788038bf6db6f51481d75cf047bddd73206/imbalanced-learn-0.2.0.tar.gz (90kB)
Requirement already satisfied: scipy>=0.17.0 in /databricks/python3/lib/python3.5/site-packages (from imbalanced-learn<0.2.1) (0.18.1)
Requirement already satisfied: numpy>=1.10.4 in /databricks/python3/lib/python3.5/site-packages (from imbalanced-learn<0.2.1) (1.11.1)
Requirement already satisfied: scikit-learn>=0.17.1 in /databricks/python3/lib/python3.5/site-packages (from imbalanced-learn<0.2.1) (0.18.1)
Building wheels for collected packages: imbalanced-learn
 Running setup.py bdist_wheel for imbalanced-learn: started
 Running setup.py bdist_wheel for imbalanced-learn: finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/9f/ae/78/2eb0580f426af24d1c23c46d169454eb4a4016cbeb80a6272d
Successfully built imbalanced-learn
Installing collected packages: imbalanced-learn
Successfully installed imbalanced-learn-0.2.0
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [7]:
%sh
pip install --upgrade pip

Collecting pip
 Downloading https://files.pythonhosted.org/packages/c2/d7/90f34cb0d83a6c5631cf71dfe64cc1054598c843a92b400e55675cc2ac37/pip-18.1-py2.py3-none-any.whl (1.3MB)
Installing collected packages: pip
 Found existing installation: pip 8.1.1
 Not uninstalling pip at /usr/lib/python2.7/dist-packages, outside environment /usr
Successfully installed pip-18.1

In [8]:
from pyspark.ml.feature import StringIndexer

categoricalColumns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
df_indexed = df

for categoricalCol in categoricalColumns:
  stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
  df_indexed = stringIndexer.fit(df_indexed).transform(df_indexed)
  
display(df_indexed)

id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,genderIndex,ever_marriedIndex,work_typeIndex,Residence_typeIndex,smoking_statusIndex
30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,Unknown,0,1.0,1.0,2.0,1.0,1.0
30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0,1.0,0.0,0.0,0.0,0.0
16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,Unknown,0,0.0,1.0,0.0,0.0,1.0
56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0,0.0,0.0,0.0,1.0,2.0
46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,Unknown,0,1.0,1.0,4.0,1.0,1.0
32257,Female,47.0,0,0,Yes,Private,Urban,210.95,50.1,Unknown,0,0.0,0.0,0.0,0.0,1.0
52800,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0,0.0,0.0,0.0,0.0,2.0
41413,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0,0.0,0.0,1.0,1.0,0.0
15266,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0,0.0,0.0,0.0,1.0,3.0
28674,Female,74.0,1,0,Yes,Self-employed,Urban,205.84,54.6,never smoked,0,0.0,0.0,1.0,0.0,0.0


In [9]:
df_new = df_indexed.select(['genderIndex',
                            'age',
                            'hypertension',
                            'heart_disease',
                            'ever_marriedIndex',
                            'work_typeIndex',
                            'Residence_typeIndex',
                            'avg_glucose_level',
                            'bmi',
                            'smoking_statusIndex',
                            'stroke'])
display(df_new)

genderIndex,age,hypertension,heart_disease,ever_marriedIndex,work_typeIndex,Residence_typeIndex,avg_glucose_level,bmi,smoking_statusIndex,stroke
1.0,3.0,0,0,1.0,2.0,1.0,95.12,18.0,1.0,0
1.0,58.0,1,0,0.0,0.0,0.0,87.96,39.2,0.0,0
0.0,8.0,0,0,1.0,0.0,0.0,110.89,17.6,1.0,0
0.0,70.0,0,0,0.0,0.0,1.0,69.04,35.9,2.0,0
1.0,14.0,0,0,1.0,4.0,1.0,161.28,19.1,1.0,0
0.0,47.0,0,0,0.0,0.0,0.0,210.95,50.1,1.0,0
0.0,52.0,0,0,0.0,0.0,0.0,77.59,17.7,2.0,0
0.0,75.0,0,1,0.0,1.0,1.0,243.53,27.0,0.0,0
0.0,32.0,0,0,0.0,0.0,1.0,77.67,32.3,3.0,0
0.0,74.0,1,0,0.0,1.0,0.0,205.84,54.6,0.0,0


In [10]:
d = df_new.toPandas()
print(d.shape)
print(d.dtypes)

(43400, 11)
genderIndex float64
age float64
hypertension int32
heart_disease int32
ever_marriedIndex float64
work_typeIndex float64
Residence_typeIndex float64
avg_glucose_level float64
bmi float64
smoking_statusIndex float64
stroke int32
dtype: object

In [11]:
import numpy
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=7)
x_val = d.values[:,0:10]
y_val = d.values[:,10]
X_res, y_res = sm.fit_sample(x_val, y_val)

feature=['genderIndex', 'age', 'hypertension', 'heart_disease', 'ever_marriedIndex', 'work_typeIndex', 'Residence_typeIndex', 'avg_glucose_level', 'bmi', 'smoking_statusIndex']
oversampled_df = pandas.DataFrame(X_res)
oversampled_df.columns = feature
oversampled_df = oversampled_df.assign(label = numpy.asarray(y_res))
oversampled_df = oversampled_df.sample(frac=1).reset_index(drop=True)

oversampling_attr = oversampled_df.values[:,0:10]
oversampling_label = oversampled_df.values[:,10]

print("oversampled_df", oversampled_df.groupby('label').size()) 


oversampled_df label
0.0 42617
1.0 42617
dtype: int64

In [12]:
oversampled_df.describe()

Out[ 12 ]: 
 genderIndex age hypertension heart_disease \
count 85234.000000 85234.000000 85234.000000 85234.000000 
mean 0.434751 55.051129 0.171879 0.136665 
std 0.455402 22.306735 0.336416 0.300534 
min 0.000000 0.080000 0.000000 0.000000 
25% 0.000000 41.000000 0.000000 0.000000 
50% 0.216931 60.000000 0.000000 0.000000 
75% 1.000000 74.658994 0.055074 0.000000 
max 2.000000 82.000000 1.000000 1.000000 

 ever_marriedIndex work_typeIndex Residence_typeIndex \
count 85234.000000 85234.000000 85234.000000 
mean 0.227057 0.736644 0.494091 
std 0.402636 0.958494 0.458533 
min 0.000000 0.000000 0.000000 
25% 0.000000 0.000000 0.000000 
50% 0.000000 0.221887 0.477928 
75% 0.208479 1.000000 1.000000 
max 1.000000 4.000000 1.000000 

 avg_glucose_level bmi smoking_statusIndex label 
count 85234.000000 85234.000000 85234.000000 85234.000000 
mean 116.748044 29.067559 1.183307 0.500000 
std 53.369023 6.526404 1.013841 0.500003 
min 55.000000 10.100000 0.000000 0.000000 
25% 78.850236 25.224461 0.000000 0.000000 
50% 95.899443 28.605038 1.000000 0.500000 
75% 136.704525 32.000000 2.000000 1.000000 
max 291.050000 97.600000 3.000000 1.000000

In [13]:
spark_df = spark.createDataFrame(oversampled_df)
spark_df.printSchema()

root
-- genderIndex: double (nullable = true)
-- age: double (nullable = true)
-- hypertension: double (nullable = true)
-- heart_disease: double (nullable = true)
-- ever_marriedIndex: double (nullable = true)
-- work_typeIndex: double (nullable = true)
-- Residence_typeIndex: double (nullable = true)
-- avg_glucose_level: double (nullable = true)
-- bmi: double (nullable = true)
-- smoking_statusIndex: double (nullable = true)
-- label: double (nullable = true)

In [14]:
from pyspark.ml.feature import VectorAssembler

stages = []
assembler = VectorAssembler(inputCols=['genderIndex',
 'age',
 'hypertension',
 'heart_disease',
 'ever_marriedIndex',
 'work_typeIndex',
 'Residence_typeIndex',
 'avg_glucose_level',
 'bmi',
 'smoking_statusIndex'],outputCol='features')

stages += [assembler]

In [15]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(spark_df)
df_pipeline = pipelineModel.transform(spark_df)

In [16]:
train,test = df_pipeline.randomSplit([0.8,0.2])
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 68182
Test Dataset Count: 17052

### Measure the impact of core number

In [18]:
# Decision Tree (maxDepth=5, impurity="gini")
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dt = DecisionTreeClassifier(labelCol='label',featuresCol='features')

In [19]:
dt_model = dt.fit(train)
dt_predictions = dt_model.transform(test)
dt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_acc = dt_acc_evaluator.evaluate(dt_predictions)
print(dt_acc)

0.8609547267182736

In [20]:
# Random Forest (maxDepth=5, impurity="gini", numTrees=20, subsamplingRate=1.0)
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')

In [21]:
rf_model = rf.fit(train)
rf_predictions = rf_model.transform(test)
rf_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_acc = rf_acc_evaluator.evaluate(rf_predictions)
print(rf_acc)

0.8874618813042459

In [22]:
# Gradient-Boosted Tree Classifier (maxDepth=5, maxIter=5, subsamplingRate=1.0)

from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol = 'features', labelCol = 'label', maxIter=5)

In [23]:
gbt_model = gbt.fit(train)
gbt_predictions = gbt_model.transform(test)
gbt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt_acc = gbt_acc_evaluator.evaluate(gbt_predictions)
print(gbt_acc)

0.9062866525920713

In [24]:
# Logistic Regression Classifier (threshold=0.5, maxIter=10)

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)

In [25]:
lr_model = lr.fit(train)
lr_predictions = lr_model.transform(test)
lr_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_acc = lr_acc_evaluator.evaluate(lr_predictions)
print(lr_acc)

0.7838376730002345

### Measure the impact of scale

In [27]:
tr1 = train.limit(15000)
tr2 = train.limit(30000)
tr3 = train.limit(45000)
tr4 = train.limit(60000)

In [28]:
# Decision Tree (maxDepth=5, impurity="gini")
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dt = DecisionTreeClassifier(labelCol='label',featuresCol='features')

In [29]:
dt_model = dt.fit(tr1)
dt_predictions = dt_model.transform(test)
dt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_acc = dt_acc_evaluator.evaluate(dt_predictions)
print(dt_acc)

0.8573187895847995

In [30]:
dt_model = dt.fit(tr2)
dt_predictions = dt_model.transform(test)
dt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_acc = dt_acc_evaluator.evaluate(dt_predictions)
print(dt_acc)

0.8626554069903823

In [31]:
dt_model = dt.fit(tr3)
dt_predictions = dt_model.transform(test)
dt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_acc = dt_acc_evaluator.evaluate(dt_predictions)
print(dt_acc)

0.8733286418015482

In [32]:
dt_model = dt.fit(tr4)
dt_predictions = dt_model.transform(test)
dt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_acc = dt_acc_evaluator.evaluate(dt_predictions)
print(dt_acc)

0.8703964344358434

In [33]:
# Random Forest (maxDepth=5, impurity="gini", numTrees=20, subsamplingRate=1.0)
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')

In [34]:
rf_model = rf.fit(tr1)
rf_predictions = rf_model.transform(test)
rf_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_acc = rf_acc_evaluator.evaluate(rf_predictions)
print(rf_acc)

0.8754984752521698

In [35]:
rf_model = rf.fit(tr2)
rf_predictions = rf_model.transform(test)
rf_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_acc = rf_acc_evaluator.evaluate(rf_predictions)
print(rf_acc)

0.8858784893267652

In [36]:
rf_model = rf.fit(tr3)
rf_predictions = rf_model.transform(test)
rf_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_acc = rf_acc_evaluator.evaluate(rf_predictions)
print(rf_acc)

0.8848228946751114

In [37]:
rf_model = rf.fit(tr4)
rf_predictions = rf_model.transform(test)
rf_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_acc = rf_acc_evaluator.evaluate(rf_predictions)
print(rf_acc)

0.8801900070372977

In [38]:
# Gradient-Boosted Tree Classifier (maxDepth=5, maxIter=5, subsamplingRate=1.0)

from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol = 'features', labelCol = 'label', maxIter=5)

In [39]:
gbt_model = gbt.fit(tr1)
gbt_predictions = gbt_model.transform(test)
gbt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt_acc = gbt_acc_evaluator.evaluate(gbt_predictions)
print(gbt_acc)

0.8963757916959887

In [40]:
gbt_model = gbt.fit(tr2)
gbt_predictions = gbt_model.transform(test)
gbt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt_acc = gbt_acc_evaluator.evaluate(gbt_predictions)
print(gbt_acc)

0.905524278676988

In [41]:
gbt_model = gbt.fit(tr3)
gbt_predictions = gbt_model.transform(test)
gbt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt_acc = gbt_acc_evaluator.evaluate(gbt_predictions)
print(gbt_acc)

0.9088669950738916

In [42]:
gbt_model = gbt.fit(tr4)
gbt_predictions = gbt_model.transform(test)
gbt_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt_acc = gbt_acc_evaluator.evaluate(gbt_predictions)
print(gbt_acc)

0.9184259910860896

In [43]:
# Logistic Regression Classifier (threshold=0.5, maxIter=10)

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)

In [44]:
lr_model = lr.fit(tr1)
lr_predictions = lr_model.transform(test)
lr_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_acc = lr_acc_evaluator.evaluate(lr_predictions)
print(lr_acc)

0.7743373211353507

In [45]:
lr_model = lr.fit(tr2)
lr_predictions = lr_model.transform(test)
lr_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_acc = lr_acc_evaluator.evaluate(lr_predictions)
print(lr_acc)

0.774513253577293

In [46]:
lr_model = lr.fit(tr3)
lr_predictions = lr_model.transform(test)
lr_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_acc = lr_acc_evaluator.evaluate(lr_predictions)
print(lr_acc)

0.77275392915787

In [47]:
lr_model = lr.fit(tr4)
lr_predictions = lr_model.transform(test)
lr_acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_acc = lr_acc_evaluator.evaluate(lr_predictions)
print(lr_acc)

0.7733990147783252